In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys
from pathlib import Path
import random
import datetime

from functools import reduce

import time 
from IPython.display import clear_output
import math

In [ ]:
# libraries required to use PAM (https://github.com/arup-group/pam)

import os
import numpy as np
from random import choice
import pandas as pd
import geopandas as gp
from matplotlib import pyplot as plt
%matplotlib inline
from shapely.geometry import Point

from pam.core import Population, Household, Person
from pam.activity import Plan, Activity, Leg
from pam.utils import minutes_to_datetime as mtdt
from pam.variables import END_OF_DAY
from pam.write import write_matsim, write_od_matrices, write_duration_benchmark, write_distance_benchmark, to_csv
from pam.plot.stats import plot_activity_times, plot_leg_times
from pam.read import load_travel_diary, read_matsim
from pam.samplers import facility
from pam.write import write_matsim

In [ ]:
#Read CSV file containing the agents from SPENSER
df_persons_SPENSER__dir = r'' # use your path
df_persons_SPENSER__file = os.path.join(df_persons_SPENSER__dir, "")
df_persons_SPENSER = pd.read_csv(df_persons_SPENSER__file, index_col=None, header=0)

In [ ]:
df_persons_SPENSER.head()

In [ ]:
len(df_persons_SPENSER)

In [ ]:
df_persons_SPENSER['Car_access'].unique().tolist()

In [ ]:
#Read CSV file containing the agents from SPENSER
df_agents_activities__dir = r'' # use your path
df_agents_activities__file = os.path.join(df_agents_activities__dir, "")
df_agents_activities = pd.read_csv(df_agents_activities__file, index_col=None, header=0)

In [ ]:
df_agents_activities.head()

In [ ]:
df_agents_activities['Ending_time'].max()

In [ ]:
df_agents_activities.dtypes

In [ ]:
df_agents_activities.loc[(df_agents_activities['PID_AreaMSOA'] == "107585_E02001697")]

In [ ]:
unique_agents_ID_list = df_agents_activities['PID_AreaMSOA'].unique().tolist()

In [ ]:
len(unique_agents_ID_list)

In [ ]:
df_agents_activities['Trip_duration'] = df_agents_activities['Ending_time'] - df_agents_activities['Starting_time']

In [ ]:
(df_agents_activities.loc[(df_agents_activities['Trip_duration'] < 0)])

In [ ]:
df_agents_activities['Transport_mode'].unique().tolist()

In [ ]:
public_transport_modes_list = ['train','bus','metro']

In [ ]:
# Change different public transport modes to 'pt'
df_agents_activities.loc[df_agents_activities['Transport_mode'].isin(public_transport_modes_list), 'Transport_mode'] = 'pt'

In [ ]:
# Change motorcycle mode to car
df_agents_activities.loc[df_agents_activities['Transport_mode'] == "motorcycle", 'Transport_mode'] = 'car'

In [ ]:
# Check if results are OK
df_agents_activities['Transport_mode'].unique().tolist()

In [ ]:
# Select only those agents that interact with the TW region
df_persons_SPENSER_to_use = df_persons_SPENSER.loc[(df_persons_SPENSER['PID_AreaMSOA'].isin(unique_agents_ID_list))]

In [ ]:
len(df_persons_SPENSER_to_use)

In [ ]:
########################################################################################################

In [ ]:
df_persons_SPENSER_to_use.dtypes

In [ ]:
low_income = ['group_1','group_2']
medium_income = ['group_3', 'group_4']
high_income = ['group_5']
child_no_income = ['group_0']

In [ ]:
############################ NEW START ###############################################

In [ ]:
# Spilts persons Dataframe into MSOA areas (whole households are stored in OA areas)
grps_persons_msoa_split = df_persons_SPENSER_to_use.groupby('Area_MSOA')


In [ ]:
msoa_num = 0
msoa_tot = df_persons_SPENSER_to_use.Area_MSOA.nunique()

In [ ]:
t0 = time.time()
agents_processsed = 0

counter = 0
total_people = len(df_persons_SPENSER)


for msoa_name, persons_grp in grps_persons_msoa_split:

    msoa_num += 1
    
    persons_grp = persons_grp.sort_values('HID_AreaOA')
    
        
    # Get the agents ID that are located in the MSOA level selected in each iteration of the for loop
    agents_id_in_mosa_list = persons_grp['PID_AreaMSOA'].unique().tolist()
    
    # Select only those acitivity plans related to those agents selected before
    df_activity_plans_coord_msoa_level = df_agents_activities.loc[(df_agents_activities['PID_AreaMSOA'].isin(agents_id_in_mosa_list))] 

    # loop through the persons in the MSOA level in the loop
    for idx_person, person_row in persons_grp.iterrows():
        
        
        agents_processsed +=1

        percentage_processed = round(agents_processsed/len(df_persons_SPENSER_to_use) * 100,2)
        

        counter += 1
        t1 = time.time()
        clear_output(wait=True)
        print(f'MSOA num: {msoa_num} of {msoa_tot}, MSOA name: {msoa_name}')
        print(f'Agent num: {agents_processsed} of {len(df_persons_SPENSER_to_use)}')
        print(f'time running {(t1 - t0)/3600}')
        print(f'Percentage done: {percentage_processed}')



        # Create  personal attributes
        spenser_id = person_row['PID_AreaMSOA']
        driven_license = person_row['Driving_license']
        car_access = person_row['Car_access']
        age = person_row['Age']
        
        
        #Get the trip of the SPENSER agent
        df_activity_plans_individual = df_activity_plans_coord_msoa_level.loc[(df_activity_plans_coord_msoa_level['PID_AreaMSOA'] == spenser_id)] 

        df_activity_plans_individual = df_activity_plans_individual.reset_index(drop=True)
        
        # Get the transport modes used by the agent
        transport_modes_list = df_activity_plans_individual['Transport_mode'].unique().tolist()
        
        
        if (('car' in transport_modes_list) & (car_access == False) & (age > 18)):
            
            # Set the new value to True
            car_access = True
            driven_license = True

            # Update the value in the agent
            df_persons_SPENSER_to_use.at[idx_person,'Car_access'] = car_access
            df_persons_SPENSER_to_use.at[idx_person,'Driving_license'] = driven_license
            

In [ ]:
###########################################################################

In [ ]:
# Spilts persons Dataframe into MSOA areas (whole households are stored in OA areas)
grps_persons_msoa_split = df_persons_SPENSER_to_use.groupby('Area_MSOA')


In [ ]:
len(grps_persons_msoa_split)

In [ ]:
msoa_num = 0
msoa_tot = df_persons_SPENSER_to_use.Area_MSOA.nunique()

In [ ]:
population = Population() # Initialise an empty population

#legs = []
#activities = []
t0 = time.time()
agents_processsed = 0

counter = 0
total_people = len(df_persons_SPENSER)


for msoa_name, persons_grp in grps_persons_msoa_split:

    msoa_num += 1
    
    persons_grp = persons_grp.sort_values('HID_AreaOA')
    
        
    # Get the agents ID that are located in the MSOA level selected in each iteration of the for loop
    agents_id_in_mosa_list = persons_grp['PID_AreaMSOA'].unique().tolist()
    
    # Select only those acitivity plans related to those agents selected before
    df_activity_plans_coord_msoa_level = df_agents_activities.loc[(df_agents_activities['PID_AreaMSOA'].isin(agents_id_in_mosa_list))] 

    # loop through the persons in the MSOA level in the loop
    for idx_person, person_row in persons_grp.iterrows():
        
        
        agents_processsed +=1

        percentage_processed = round(agents_processsed/len(df_persons_SPENSER_to_use) * 100,2)
        

        counter += 1
        t1 = time.time()
        clear_output(wait=True)
        print(f'MSOA num: {msoa_num} of {msoa_tot}, MSOA name: {msoa_name}')
        print(f'Agent num: {agents_processsed} of {len(df_persons_SPENSER_to_use)}')
        print(f'time running {(t1 - t0)/3600}')
        print(f'Percentage done: {percentage_processed}')



        # Create  personal attributes
        spenser_id = person_row['PID_AreaMSOA']
        gender = person_row['Sex']
        age = person_row['Age']
        marital_status = person_row['Marital_status']
        children_dependency =person_row['Children_dependency']
        economic_activity = person_row['Economic_activity']
        occupation = person_row['Occupation']
        income = person_row['Income_group']
        driving_license = person_row['Driving_license']
        car_access = person_row['Car_access']
        
        #car_access_str = str(car_access)
        
        if (car_access == False):
            carAvail = 'never'
        else:
            carAvail = 'yes'
            

 
        bike_access = person_row['Bike_access']
        hh_oa_level = person_row['Area_OA']

        # Create different agents and household   
        hh_id = str(person_row['HID_AreaOA'])
        hh = Household(str(hh_id) + '_' + str(counter))


        # Subpopulation category based on income value
        if (income in low_income):
            subpopulation = 'low_income'
        if (income in medium_income):
            subpopulation = 'medium_income'
        if (income in high_income):
            subpopulation = 'high_income'
        if (income in child_no_income):
            subpopulation = 'child_no_income'

        agent = Person(spenser_id, 
                        freq = 1,
                        attributes = {"subpopulation": subpopulation,
                                      "gender" : gender,
                                      "age": age,
                                      "marital_status": marital_status,
                                      "children_dependency": children_dependency,
                                      "economic_activity": economic_activity,
                                      "occupation": occupation,
                                      "income": income,
                                      "driving_license": driving_license,
                                      "car_access": car_access,
                                      "carAvail": carAvail,
                                      "bike_access": bike_access,
                                      "household_ID": hh_id,
                                      "household_OA_level": hh_oa_level
                                     })

        hh.add(agent)
        population.add(hh)

        #Get the trip of the SPENSER agent
        df_activity_plans_individual = df_activity_plans_coord_msoa_level.loc[(df_activity_plans_coord_msoa_level['PID_AreaMSOA'] == spenser_id)] 

        df_activity_plans_individual = df_activity_plans_individual.reset_index(drop=True)


        # Get the number of trips 
        number_of_legs = len(df_activity_plans_individual)


        ###############################################################
        # Define the first activity

        ## Home location:
        ### Always the last row of the activities belongs to home, so the coordinates can be obtaned from there
        home_location_x = int(df_activity_plans_individual.iloc[-1]['Easting'])
        home_location_y = int(df_activity_plans_individual.iloc[-1]['Northing']) #'Y column name'
        home_location_geom = Point(home_location_x, home_location_y)

        # Get the ending time of the first actvitivy, which is the starting time of the first row of the 'df_activity_plans_individual'
        leg_starting_time = int(df_activity_plans_individual.iloc[0]['Starting_time']) #'Start column name'
        leg_ending_time = int(df_activity_plans_individual.iloc[0]['Ending_time']) #'end column name'
        
        
        if (leg_starting_time == 0):
            
            
            leg_starting_time == int(450) # first leg will end at 8am
            
            
            
            if (leg_ending_time != 0):
                
                if (leg_starting_time > leg_ending_time):
                    
                    leg_starting_time = leg_ending_time - 15

            # Update the value in the dataframe
            df_activity_plans_individual.at[0,'Starting_time'] = int(leg_starting_time)
        
        if (leg_ending_time == 0):

            leg_ending_time == leg_starting_time + int(30) # first leg will last for 30 min

            # Update the value in the dataframe
            df_activity_plans_individual.at[0,'Ending_time'] = int(leg_ending_time)

        
        
        
        activity_duration_seconds = int(round(leg_starting_time*60,0))
        
        activity_duration_15min_block = int(activity_duration_seconds/900)
        
        activity_duration_15min_block_in_seconds = activity_duration_15min_block * 15 * 60
        
        # If the length of the activity is set to 0, update the value to 1800seconds (30min)
        if (activity_duration_15min_block_in_seconds == 0):
            
            activity_duration_15min_block_in_seconds = 1800
            
        # If the length of the activity is below 0, update the value to 900seconds (15min)
        if (activity_duration_15min_block_in_seconds < 0):
            
            activity_duration_15min_block_in_seconds = 900
        
        
        # Save the informtaion in the agent object
        agent.add(Activity(seq = 1,
                          act = "home" + "_" +str(activity_duration_15min_block_in_seconds),
                          loc = home_location_geom,
                          start_time = mtdt(0),
                          end_time = mtdt(leg_starting_time)
                          ))

        ###############################################################

        # loop through the activity plan and define the activities (starting from second) and legs
        for i in range(number_of_legs):

            #print('trip:')
            #print( i + 1)
            # Select mode, purpose of trip (activity), starting and ending times and locations
            transport = df_activity_plans_individual.iloc[i]['Transport_mode'] #'Transport mode column name'
            
            # Fix the possible issue of having children driving a car:
            # In that case, the transport mode will be changed to car_passenger
            if ((transport == 'car') & (age < 18)):
                transport = 'car_passenger'
       
            
            activity =  df_activity_plans_individual.iloc[i]['Trip_purpose'] #'Purpose of the trip column name'
            
            # Update the value of education in case it is only named as "education"
            if (activity == "education"):
                activity = "education_0_15"
            
            leg_starting_time = int(df_activity_plans_individual.iloc[i]['Starting_time']) #'Start column name'
            leg_ending_time = int(df_activity_plans_individual.iloc[i]['Ending_time']) #'end column name'
            

            ###############################################################
            # if first trip has not starting and ending times:
            if ((leg_starting_time == 0) and (leg_ending_time == 0) and (i== 0)):  

                if (df_activity_plans_individual.iloc[i]['Trip_purpose'] == 'work'):
                    
                    leg_starting_time = int(450)
                    
                    leg_ending_time = int(leg_starting_time + int(30))
            
            
                # upate the value in the dataframe
                df_activity_plans_individual.at[i,'Starting_time'] = int(leg_starting_time)

                # upate the value in the dataframe
                df_activity_plans_individual.at[i,'Ending_time'] = int(leg_ending_time)
            
            ###############################################################
            # if first trip has not starting and ending times:
            if ((leg_ending_time == 0) and (i==0)): 
                
                
                if (df_activity_plans_individual.iloc[i]['Trip_purpose'] == 'work'):
                    
                    # if work started before 9:30am:
                    if (df_activity_plans_individual.iloc[i-1]['Ending_time'] < 540):
                    
                        leg_ending_time = int(leg_starting_time + int(420))
                        
                    else:
                        
                        leg_ending_time = int(leg_starting_time + int(210))
            
                else:

                    leg_ending_time = int(leg_starting_time + int(30))

                # upate the value in the dataframe
                df_activity_plans_individual.at[i,'Ending_time'] = int(leg_ending_time)
            
            
            ###############################################################
            
            # if any leg except last one has not starting and ending times, when there are only two trips
            if ((leg_starting_time == 0) and (leg_ending_time == 0)):  
            
                if (df_activity_plans_individual.iloc[i-1]['Trip_purpose'] == 'work'):
        
                    # if work started before 9:30am:
                    if (df_activity_plans_individual.iloc[i-1]['Ending_time'] < 540):
                        # work activity will take 7hours
                        leg_starting_time = int(df_activity_plans_individual.iloc[i-1]['Ending_time'] + int(420))

                    else:
                        # work activity will take 3.5hours
                        leg_starting_time = int(df_activity_plans_individual.iloc[i-1]['Ending_time'] + int(210))
                
                
                else:
                    # activity will take 0.5hours
                    leg_starting_time = int(df_activity_plans_individual.iloc[i-1]['Ending_time'] + int(30)) # stay at place for 30 min only
                
                ###### Get the ending time of the leg:
                leg_ending_time = int(leg_starting_time + int(30))
                
                #print('leg ending time')
                #print(leg_ending_time)
                
                if (leg_starting_time > 1400):
            
                    leg_starting_time = 1400

                    leg_ending_time = 1430

                if (leg_ending_time > 1400):

                    leg_starting_time = 1400

                    leg_ending_time = 1430

                #upate the value in the dataframe#
                df_activity_plans_individual.at[i,'Starting_time'] = int(leg_starting_time)  

                # upate the value in the dataframe#
                df_activity_plans_individual.at[i,'Ending_time'] = int(leg_ending_time)

            ###############################################################
            
            # if any leg except last one has not starting and ending times, when there are only two trips
            if ((leg_starting_time == 0) and (leg_ending_time != 0) and (i != 0)):  
            
                if (df_activity_plans_individual.iloc[i-1]['Trip_purpose'] == 'work'):
        
                    # if work started before 9:30am:
                    if (df_activity_plans_individual.iloc[i-1]['Ending_time'] < 540):
                        # work activity will take 7hours
                        leg_starting_time = int(df_activity_plans_individual.iloc[i-1]['Ending_time'] + 420)

                    else:
                        # work activity will take 3.5hours
                        leg_starting_time = int(df_activity_plans_individual.iloc[i-1]['Ending_time'] + 210)
                
                
                else:
                                      
                    
                    # activity will take 0.5hours
                    leg_starting_time = int(df_activity_plans_individual.iloc[i-1]['Ending_time'] + 30) # stay at place for 30 min only
                
                
                if (leg_starting_time > leg_ending_time):
            
                    leg_starting_time = int(leg_ending_time - int(30))

                #upate the value in the dataframe#
                df_activity_plans_individual.at[i,'Starting_time'] = int(leg_starting_time)  
            
            ###############################################################
            
            # if any leg except last one has not starting and ending times, when there are only two trips
            if ((leg_starting_time != 0) and (leg_ending_time == 0)):  
            
                ###### Get the ending time of the leg:
                leg_ending_time = int(leg_starting_time + int(30))
                
                if (leg_ending_time > 1400):

                    leg_ending_time = 1430

                # upate the value in the dataframe#
                df_activity_plans_individual.at[i,'Ending_time'] = int(leg_ending_time)
            
        
            ###############################################################

            # Origin-Destination Location: depending if it is the first activity or the rest:
            if (i == 0):
                # Origin is home
                origin_location_geom = home_location_geom

                destination_location_x = int(df_activity_plans_individual.iloc[i]['Easting']) #'X column name'
                destination_location_y = int(df_activity_plans_individual.iloc[i]['Northing']) #'Y column name'

            else:
                # Select the location where previous activity finished
                origin_location_x = int(df_activity_plans_individual.iloc[i-1]['Easting']) #'X column name'
                origin_location_y = int(df_activity_plans_individual.iloc[i-1]['Northing']) #'Y column name'
                origin_location_geom = Point(origin_location_x, origin_location_y)

                destination_location_x = int(df_activity_plans_individual.iloc[i]['Easting']) #'X column name'
                destination_location_y = int(df_activity_plans_individual.iloc[i]['Northing']) #'Y column name'

            destination_location_geom = Point(destination_location_x, destination_location_y)

            ###############################################################

            # Create the legs
            agent.add(Leg(seq = i + 1,
                          mode = transport,
                          start_loc = origin_location_geom,
                          end_loc = destination_location_geom,
                          start_time = mtdt(leg_starting_time),
                          end_time = mtdt(leg_ending_time),
                         ))

            ###############################################################

            # Create the activities
            if (i + 1 < number_of_legs):

                activity_ending_time = int(df_activity_plans_individual.iloc[i + 1]['Starting_time']) #'starting time of the next leg'


                if (activity_ending_time == 0):

                    if (activity == 'work'):

                        if (leg_starting_time < 510):

                            activity_ending_time = int(leg_ending_time + 420) # stay at workplace for 7hours

                        else:
                            activity_ending_time = int(leg_ending_time + 210) # stay at workplace for 3.5hours

                    else:
                        activity_ending_time = int(leg_ending_time + 30) # stay at place for 30 min only

                    df_activity_plans_individual.at[i+1,'Starting_time'] = int(activity_ending_time)
                    
                    
                activity_duration_seconds = int(round((activity_ending_time - leg_ending_time)*60,0))
                
                activity_duration_15min_block = int(activity_duration_seconds/900)
        
                activity_duration_15min_block_in_seconds = activity_duration_15min_block * 15 * 60
        
                # If the length of the activity is set to 0, update the value to 1800seconds (30min)
                if (activity_duration_15min_block_in_seconds == 0):
            
                    activity_duration_15min_block_in_seconds = 900
                
                # If the length of the activity is below 0, update the value to 900seconds (15min)
                if (activity_duration_15min_block_in_seconds < 0):
            
                    activity_duration_15min_block_in_seconds = 900


                ###################################

                agent.add(Activity(seq = i + 2,
                          act = activity + "_" +str(activity_duration_15min_block_in_seconds),
                          loc = destination_location_geom,
                          start_time = mtdt(leg_ending_time),
                          end_time = mtdt(activity_ending_time)
                         ))

            if (i + 1 == number_of_legs):
                
                activity_duration_seconds = int(round((1440 - leg_ending_time)*60,0))
                
                activity_duration_15min_block = int(activity_duration_seconds/900)
        
                activity_duration_15min_block_in_seconds = activity_duration_15min_block * 15 * 60
        
                # If the length of the activity is set to 0, update the value to 1800seconds (30min)
                if (activity_duration_15min_block_in_seconds == 0):
            
                    activity_duration_15min_block_in_seconds = 900
        

                # If the length of the activity is below 0, update the value to 900seconds (15min)
                if (activity_duration_15min_block_in_seconds < 0):
            
                    activity_duration_15min_block_in_seconds = 900
                
                
            
                agent.add(Activity(seq = i + 2,
                                   act = activity + "_" +str(activity_duration_15min_block_in_seconds),
                                   loc = home_location_geom,
                                   start_time = mtdt(leg_ending_time),
                                   end_time = END_OF_DAY
                                  ))


            
print('Code has finishes. Check results, amigo!')


In [ ]:
#write_matsim(population=population, plans_path=os.path.join("outputs", "population_20230515_carAvail_fix.xml"), version=12)

In [ ]:
print(population.stats)

In [ ]:
#hh = population.random_household()
#hh.print()
#hh.plot()

In [ ]:
for i in range (72):
    
    print(i)
    population.random_person().print()

In [ ]:
population.print()

In [ ]:
population.activity_classes

In [ ]:
population.mode_classes

In [ ]:
population.random_person().print()

In [ ]:
# Validation if it works
population.fix_plans()

In [ ]:
# Validation if it works
population.validate()

In [ ]:
write_matsim(population=population, plans_path=os.path.join("outputs", "population_20230516_VALIDATED_PAM_carAllowedToThoseUsingIt.xml"), version=12)

In [ ]:
population.activity_classes

In [ ]:
print(population.stats)

In [ ]:
#######################################################################

#                             analysis                                #


#######################################################################

In [ ]:
# Check the duration of trips
durations = write_duration_benchmark(population)
durations

In [ ]:
plt.barh(durations["duration"], durations["trips"])
plt.xlabel("Counts")
plt.ylabel("Duration for trips")
plt.title("Duration for different trips")
plt.ylim(ymax = "90 to 120 min")

In [ ]:
# Check the distance of trips
distances  = write_distance_benchmark(population)
distances

In [ ]:
plt.barh(distances["distance"], distances["trips"])
plt.xlabel("Counts")
plt.ylabel("distance, km")
plt.title("distance for different trips")
plt.ylim(ymax = "200+ km")

In [ ]:
to_csv(population, dir = "outputs", crs = "epsg:27700")

In [ ]:
df_activity = pd.read_csv(os.path.join("outputs", "activities.csv"))
totals = df_activity.activity.value_counts()
plt.barh(totals.index, totals)
plt.title("activities count")


In [ ]:
people = pd.read_csv(os.path.join("outputs", "people.csv")).set_index('pid')
hhs = pd.read_csv(os.path.join("outputs", "households.csv")).set_index('hid')
trips = pd.read_csv(os.path.join("outputs", "legs.csv")).drop(['Unnamed: 0'],axis=1)

In [ ]:
trips.head()

In [ ]:
##############################################################################################

In [ ]:
# Check the average trips per gender

In [ ]:
df_persons_SPENSER_to_use_MALES = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 1)]

persons_SPENSER_to_use_MALES_list = df_persons_SPENSER_to_use_MALES['PID_AreaMSOA'].unique().tolist()

number_trips_MALES = len(trips.loc[trips['pid'].isin(persons_SPENSER_to_use_MALES_list)])

print(number_trips_MALES)

In [ ]:
df_persons_SPENSER_to_use_FEMALES = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 2)]

persons_SPENSER_to_use_FEMALES_list = df_persons_SPENSER_to_use_FEMALES['PID_AreaMSOA'].unique().tolist()

number_trips_FEMALES = len(trips.loc[trips['pid'].isin(persons_SPENSER_to_use_FEMALES_list)])

print(number_trips_FEMALES)

In [ ]:
# NTS 2019
## Ratio trips per year males (915) and females (990)
990/915

In [ ]:
number_trips_FEMALES/number_trips_MALES

In [ ]:
##############################################################################################

In [ ]:
# Check the average commuting trips per gender and range of age

In [ ]:
df_persons_SPENSER_to_use_MALES_17_20 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 1) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 17) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 20)]

persons_SPENSER_to_use_MALES_17_20_list = df_persons_SPENSER_to_use_MALES_17_20['PID_AreaMSOA'].unique().tolist()

number_trips_MALES_17_20 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_MALES_17_20_list)) &
                                         (trips['destination activity'] == 'work')])


#############################
df_persons_SPENSER_to_use_MALES_21_29 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 1) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 21) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 29) ]

persons_SPENSER_to_use_MALES_21_29_list = df_persons_SPENSER_to_use_MALES_21_29['PID_AreaMSOA'].unique().tolist()

number_trips_MALES_21_29 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_MALES_21_29_list)) &
                                         (trips['destination activity'] == 'work')])


#############################
df_persons_SPENSER_to_use_MALES_30_39 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 1) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 30) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 39) ]

persons_SPENSER_to_use_MALES_30_39_list = df_persons_SPENSER_to_use_MALES_30_39['PID_AreaMSOA'].unique().tolist()

number_trips_MALES_30_39 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_MALES_30_39_list)) &
                                         (trips['destination activity'] == 'work')])

#############################
df_persons_SPENSER_to_use_MALES_40_49 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 1) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 40) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 49) ]

persons_SPENSER_to_use_MALES_40_49_list = df_persons_SPENSER_to_use_MALES_40_49['PID_AreaMSOA'].unique().tolist()


number_trips_MALES_40_49 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_MALES_40_49_list)) &
                                         (trips['destination activity'] == 'work')])

#############################
df_persons_SPENSER_to_use_MALES_50_59 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 1) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 50) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 59) ]

persons_SPENSER_to_use_MALES_50_59_list = df_persons_SPENSER_to_use_MALES_50_59['PID_AreaMSOA'].unique().tolist()

number_trips_MALES_50_59 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_MALES_50_59_list)) &
                                         (trips['destination activity'] == 'work')])

#############################
df_persons_SPENSER_to_use_MALES_60_69 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 1) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 60) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 69) ]

persons_SPENSER_to_use_MALES_60_69_list = df_persons_SPENSER_to_use_MALES_60_69['PID_AreaMSOA'].unique().tolist()

number_trips_MALES_60_69 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_MALES_60_69_list)) &
                                         (trips['destination activity'] == 'work')])


#############################
df_persons_SPENSER_to_use_MALES_70_120 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 1) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 70) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 120) ]

persons_SPENSER_to_use_MALES_70_120_list = df_persons_SPENSER_to_use_MALES_70_120['PID_AreaMSOA'].unique().tolist()

number_trips_MALES_70_120 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_MALES_70_120_list)) &
                                         (trips['destination activity'] == 'work')])

#############################


In [ ]:
df_persons_SPENSER_to_use_FEMALES_17_20 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 2) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 17) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 20)]

persons_SPENSER_to_use_FEMALES_17_20_list = df_persons_SPENSER_to_use_FEMALES_17_20['PID_AreaMSOA'].unique().tolist()

number_trips_FEMALES_17_20 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_FEMALES_17_20_list)) &
                                         (trips['destination activity'] == 'work')])


#############################
df_persons_SPENSER_to_use_FEMALES_21_29 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 2) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 21) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 29) ]

persons_SPENSER_to_use_FEMALES_21_29_list = df_persons_SPENSER_to_use_FEMALES_21_29['PID_AreaMSOA'].unique().tolist()

number_trips_FEMALES_21_29 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_FEMALES_21_29_list)) &
                                         (trips['destination activity'] == 'work')])


#############################
df_persons_SPENSER_to_use_FEMALES_30_39 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 2) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 30) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 39) ]

persons_SPENSER_to_use_FEMALES_30_39_list = df_persons_SPENSER_to_use_FEMALES_30_39['PID_AreaMSOA'].unique().tolist()

number_trips_FEMALES_30_39 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_FEMALES_30_39_list)) &
                                         (trips['destination activity'] == 'work')])

#############################
df_persons_SPENSER_to_use_FEMALES_40_49 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 2) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 40) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 49) ]

persons_SPENSER_to_use_FEMALES_40_49_list = df_persons_SPENSER_to_use_FEMALES_40_49['PID_AreaMSOA'].unique().tolist()


number_trips_FEMALES_40_49 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_FEMALES_40_49_list)) &
                                         (trips['destination activity'] == 'work')])

#############################
df_persons_SPENSER_to_use_FEMALES_50_59 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 2) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 50) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 59) ]

persons_SPENSER_to_use_FEMALES_50_59_list = df_persons_SPENSER_to_use_FEMALES_50_59['PID_AreaMSOA'].unique().tolist()

number_trips_FEMALES_50_59 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_FEMALES_50_59_list)) &
                                         (trips['destination activity'] == 'work')])

#############################
df_persons_SPENSER_to_use_FEMALES_60_69 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 2) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 60) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 69) ]

persons_SPENSER_to_use_FEMALES_60_69_list = df_persons_SPENSER_to_use_FEMALES_60_69['PID_AreaMSOA'].unique().tolist()

number_trips_FEMALES_60_69 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_FEMALES_60_69_list)) &
                                         (trips['destination activity'] == 'work')])


#############################
df_persons_SPENSER_to_use_FEMALES_70_120 = df_persons_SPENSER_to_use.loc[(df_persons_SPENSER_to_use['Sex'] == 2) &
                                                                      (df_persons_SPENSER_to_use['Age'] >= 70) &
                                                                      (df_persons_SPENSER_to_use['Age'] <= 120) ]

persons_SPENSER_to_use_FEMALES_70_120_list = df_persons_SPENSER_to_use_FEMALES_70_120['PID_AreaMSOA'].unique().tolist()

number_trips_FEMALES_70_120 = len(trips.loc[(trips['pid'].isin(persons_SPENSER_to_use_FEMALES_70_120_list)) &
                                         (trips['destination activity'] == 'work')])

#############################


In [ ]:
print('MALES')
print(number_trips_MALES_17_20)
print(number_trips_MALES_21_29)
print(number_trips_MALES_30_39)
print(number_trips_MALES_40_49)
print(number_trips_MALES_50_59)
print(number_trips_MALES_60_69)
print(number_trips_MALES_70_120)


print('\nFEMALES')
print(number_trips_FEMALES_17_20)
print(number_trips_FEMALES_21_29)
print(number_trips_FEMALES_30_39)
print(number_trips_FEMALES_40_49)
print(number_trips_FEMALES_50_59)
print(number_trips_FEMALES_60_69)
print(number_trips_FEMALES_70_120)


In [ ]:
##############################################################################################

In [ ]:
# Trips by transport mode

In [ ]:
trips.groupby(['mode'], as_index=False).sum()

In [ ]:
trips_by_bike = len(trips.loc[trips['mode'] == 'bike'])
trips_by_bus = len(trips.loc[trips['mode'] == 'bus'])
trips_by_car = len(trips.loc[trips['mode'] == 'car'])
trips_by_car_passenger = len(trips.loc[trips['mode'] == 'car_passenger'])
trips_by_metro = len(trips.loc[trips['mode'] == 'metro'])
trips_by_motocycle = len(trips.loc[trips['mode'] == 'motocycle'])
trips_by_train = len(trips.loc[trips['mode'] == 'train'])
trips_by_walk = len(trips.loc[trips['mode'] == 'walk'])

In [ ]:
total_trips = trips_by_bike + trips_by_bus + trips_by_car + trips_by_car_passenger + trips_by_metro + trips_by_motocycle + trips_by_train + trips_by_walk

In [ ]:
percentage_trips_by_bike = round(trips_by_bike/total_trips *100,2)
percentage_trips_by_bus = round(trips_by_bus/total_trips *100,2)
percentage_trips_by_car = round(trips_by_car/total_trips *100,2)
percentage_trips_by_car_passenger = round(trips_by_car_passenger/total_trips *100,2)
percentage_trips_by_metro = round(trips_by_metro/total_trips *100,2)
percentage_trips_by_motocycle = round(trips_by_motocycle/total_trips *100,2)
percentage_trips_by_train = round(trips_by_train/total_trips *100,2)
percentage_trips_by_walk = round(trips_by_walk/total_trips *100,2)


print(percentage_trips_by_bike)
print(percentage_trips_by_bus)
print(percentage_trips_by_car)
print(percentage_trips_by_car_passenger)
print(percentage_trips_by_metro)
print(percentage_trips_by_motocycle)
print(percentage_trips_by_train)
print(percentage_trips_by_walk)

In [ ]:
##############################################################################################

In [ ]:
#Trips by purpose

In [ ]:
number_trips_work = len(trips.loc[(trips['destination activity'] == 'work')])
number_trips_education = len(trips.loc[(trips['destination activity'] == 'education')])
number_trips_education_0_15 = len(trips.loc[(trips['destination activity'] == 'education_0_15')])
number_trips_education_16_18 = len(trips.loc[(trips['destination activity'] == 'education_16_18')])
number_trips_education_18plus = len(trips.loc[(trips['destination activity'] == 'education_18plus')])
number_trips_escort_education = len(trips.loc[(trips['destination activity'] == 'escort_education')])
number_trips_escort_home = len(trips.loc[(trips['destination activity'] == 'escort_home')])
number_trips_escort_other = len(trips.loc[(trips['destination activity'] == 'escort_other')])
number_trips_escort_work = len(trips.loc[(trips['destination activity'] == 'escort_work')])
number_trips_home = len(trips.loc[(trips['destination activity'] == 'home')])
number_trips_leisure_act = len(trips.loc[(trips['destination activity'] == 'leisure_act')])
number_trips_leisure_sport = len(trips.loc[(trips['destination activity'] == 'leisure_sport')])
number_trips_medical = len(trips.loc[(trips['destination activity'] == 'medical')])
number_trips_other = len(trips.loc[(trips['destination activity'] == 'other')])
number_trips_shop = len(trips.loc[(trips['destination activity'] == 'shop')])
number_trips_supermarket = len(trips.loc[(trips['destination activity'] == 'supermarket')])
number_trips_eat = len(trips.loc[(trips['destination activity'] == 'eat')])


In [ ]:
total_trips = number_trips_work + number_trips_education_0_15 + perc_number_trips_education + number_trips_education_16_18 + number_trips_education_18plus + number_trips_escort_education + number_trips_escort_home + number_trips_escort_other + number_trips_escort_work + number_trips_leisure_act + number_trips_leisure_sport + number_trips_medical + number_trips_other + number_trips_shop + number_trips_supermarket + number_trips_eat

In [ ]:
perc_number_trips_work = round(number_trips_work/total_trips*100,2)
perc_number_trips_education = round(number_trips_education/total_trips*100,2)
perc_number_trips_education_0_15 = round(number_trips_education_0_15/total_trips*100,2)
perc_number_trips_education_16_18 = round(number_trips_education_16_18/total_trips*100,2)
perc_number_trips_education_18plus = round(number_trips_education_18plus/total_trips*100,2)
perc_number_trips_escort_education = round(number_trips_escort_education/total_trips*100,2)
perc_number_trips_escort_home = round(number_trips_escort_home/total_trips*100,2)
perc_number_trips_escort_other = round(number_trips_escort_other/total_trips*100,2)
perc_number_trips_escort_work = round(number_trips_escort_work/total_trips*100,2)
perc_number_trips_leisure_act = round(number_trips_leisure_act/total_trips*100,2)
perc_number_trips_leisure_sport = round(number_trips_leisure_sport/total_trips*100,2)
perc_number_trips_medical = round(number_trips_medical/total_trips*100,2)
perc_number_trips_other = round(number_trips_other/total_trips*100,2)
perc_number_trips_shop = round(number_trips_shop/total_trips*100,2)
perc_number_trips_supermarket = round(number_trips_supermarket/total_trips*100,2)
perc_number_trips_eat = round(number_trips_eat/total_trips*100,2)


In [ ]:
print(perc_number_trips_work)
print(perc_number_trips_education)
print(perc_number_trips_education_0_15)
print(perc_number_trips_education_16_18)
print(perc_number_trips_education_18plus)
print(perc_number_trips_escort_education)
print(perc_number_trips_escort_home)
print(perc_number_trips_escort_other)
print(perc_number_trips_escort_work)
print(perc_number_trips_leisure_act)
print(perc_number_trips_leisure_sport)
print(perc_number_trips_medical)
print(perc_number_trips_other)
print(perc_number_trips_shop)
print(perc_number_trips_supermarket)
print(perc_number_trips_eat)

In [ ]:
# SHOPPING
print(perc_number_trips_shop + perc_number_trips_supermarket)

In [ ]:
# LEISURE
print(perc_number_trips_eat + perc_number_trips_leisure_act + perc_number_trips_leisure_sport)


In [ ]:
# EDUCATION
print(perc_number_trips_education + perc_number_trips_education_0_15 + perc_number_trips_education_16_18 + perc_number_trips_education_18plus)

In [ ]:
# WORK
print(perc_number_trips_work)

In [ ]:
# ESCORT
print(perc_number_trips_escort_education + perc_number_trips_escort_home + perc_number_trips_escort_other + perc_number_trips_escort_work)

In [ ]:
# OTHER ESCORT
print(perc_number_trips_escort_home + perc_number_trips_escort_other + perc_number_trips_escort_work)

In [ ]:
# OTHER
print(perc_number_trips_other)

## TOO MANY OTHER ACTIVITIES THAT probaly COULD BE CONSIDERED AS leisure

In [ ]:
##############################################################################################